In [1]:
# Imports
import duckdb
import pandas as pd

# Criar conexão
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

kb_querys = [
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_chaves_na_mao.chaves_na_mao_register",
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_viva_real.viva_real_register",
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_zap_imoveis.zap_imoveis_register"
]

# Fetch knowledge base data
df_kb_list = [con.execute(query).fetch_df() for query in kb_querys]

# Concatenate knowledge base data
df_kb = pd.concat(df_kb_list, ignore_index=True)

In [8]:
import duckdb

# Criar conexão
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

# Definir a quaery
query = """
DROP SCHEMA kodomiya_leilao_imovel CASCADE;
"""

#con.execute(query)

In [11]:
import duckdb
from datetime import datetime

# Criar conexão
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

# Executar uma query no banco
today = datetime.now().strftime("%Y-%m-%d")
df_leilao = con.execute(f"SELECT * FROM kodomiya_leilao_imovel.leilao_imovel_register WHERE data_primeira_praca	< '{today}' OR data_segunda_praca < '{today}'").fetchdf()

In [9]:
df_leilao[["bairro", "rua", "cidade", "latitude", "longitude"]]

,bairro,rua,cidade,latitude,longitude
0,umbara,RUA DILSON LUIZ,Curitiba,NaN,NaN
1,uberaba,RUA RODOLFO BERNARDELLI,Curitiba,-25.489124,-49.226351


In [12]:
df_leilao["bairro"].value_counts()

bairro
umbara     1
uberaba    1
Name: count, dtype: int64

In [5]:
import duckdb
from datetime import datetime

# Criar conexão
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

# Executar uma query no banco
today = datetime.now().strftime("%Y-%m-%d")
df_leilao = con.execute(f"SELECT * FROM kodomiya_leilao_imovel.leilao_imovel_register").fetchdf()

## Drop imoveis older than oldest imovel on leilao table

In [16]:
import duckdb
from datetime import datetime

# Criar conexão
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

# Read leilao table
df_leilao = con.execute(f"SELECT * FROM kodomiya_leilao_imovel.leilao_imovel_register").fetchdf()

# Take min_date
min_date = df_leilao["datahora"].min().strftime("%Y-%m-%d")

In [24]:
del_query = f"""
    DELETE FROM kodomiya_zap_imoveis.zap_imoveis_history
    WHERE datahora < '{min_date}'
"""

# con.execute(del_query)

# Explore

In [ ]:
import duckdb
import pandas as pd

# Criar conexão
con = duckdb.connect(database=r"example.duckdb")
                  
# Select all columns needed for knowledge base
kb_querys = [
    "SELECT * FROM kodomiya_chaves_na_mao.chaves_na_mao_register",
    "SELECT * FROM kodomiya_viva_real.viva_real_register",
    "SELECT * FROM kodomiya_zap_imoveis.zap_imoveis_register"
]

# Fetch knowledge base data
df_kb_list = [con.execute(query).fetch_df() for query in kb_querys]

# Concatenate knowledge base data
df_kb = pd.concat(df_kb_list, ignore_index=True)


In [27]:
df_kb.columns

Index(['id', 'datahora', 'preco', 'tamanho', 'n_quartos', 'n_banheiros',
       'n_garagem', 'rua', 'bairro', 'cidade', 'latitude', 'longitude',
       '_dlt_load_id', '_dlt_id'],
      dtype='object')

# Standalone exploration

In [2]:
import duckdb
from unidecode import unidecode
from datetime import datetime
import logging
import os
import sys
import requests
import time
import joblib
import numpy as np
import json
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

In [3]:
SELIC_RATE_ANNUAL = 14.50 / 100  # Annual SELIC rate (14.50%)

def impute_lat_lon(df_to_impute, df_knowledge_base, logger):
    """
    Imputes missing lat/lon in a dataframe using means from a knowledge base dataframe.
    If imputing a dataframe on itself, pass it for both arguments.
    """
    logger.info(f"Starting lat/lon imputation. Nulls before: Lat={df_to_impute['latitude'].isnull().sum()}, Lon={df_to_impute['longitude'].isnull().sum()}")
    
    df = df_to_impute.copy()

    # 1. Calculate means from the knowledge base
    city_mean_lat = df_knowledge_base['latitude'].mean()
    city_mean_lon = df_knowledge_base['longitude'].mean()

    if not (pd.notna(city_mean_lat) and pd.notna(city_mean_lon)):
        logger.warning("Could not calculate city-wide mean coordinates from knowledge base. Skipping imputation.")
        df.dropna(subset=['latitude', 'longitude'], inplace=True)
        return df

    logger.info(f"Using KB for imputation. City-wide mean: Lat={city_mean_lat:.4f}, Lon={city_mean_lon:.4f}")
    bairro_lat_map = df_knowledge_base.groupby('bairro')['latitude'].mean()
    bairro_lon_map = df_knowledge_base.groupby('bairro')['longitude'].mean()
    
    # 2. Map bairro-specific means for rows with missing lat/lon
    lat_na_mask = df['latitude'].isna()
    if lat_na_mask.any():
        df.loc[lat_na_mask, 'latitude'] = df.loc[lat_na_mask, 'bairro'].map(bairro_lat_map)

    lon_na_mask = df['longitude'].isna()
    if lon_na_mask.any():
        df.loc[lon_na_mask, 'longitude'] = df.loc[lon_na_mask, 'bairro'].map(bairro_lon_map)
    
    # 3. Fallback to city-wide mean for any remaining NaNs
    df['latitude'].fillna(city_mean_lat, inplace=True)
    df['longitude'].fillna(city_mean_lon, inplace=True)

    # 4. Final drop for safety
    df.dropna(subset=['latitude', 'longitude'], inplace=True)
    
    logger.info(f"Lat/lon imputation complete. Nulls after: Lat={df['latitude'].isnull().sum()}, Lon={df['longitude'].isnull().sum()}")

    return df

def calculate_adjusted_roi(purchase_price, estimated_resale_value, logger):
    """
    Calculates a more realistic ROI by subtracting fixed and variable costs,
    including opportunity cost based on the SELIC rate.

    Args:
        purchase_price (float): The price paid for the property.
        estimated_resale_value (float): The estimated value the property will be sold for.
        logger (logging.Logger): The logger instance.

    Returns:
        float: The adjusted ROI as a percentage, or 0.0 if costs exceed profit.
    """
    try:
        # 1. Transactional Costs on Purchase
        documentation_cost = purchase_price * 0.055
        auctioneer_commission = purchase_price * 0.05
        legal_fee = 5000  # Imissão na posse
        other_costs = 300
        
        total_purchase_costs = documentation_cost + auctioneer_commission + legal_fee + other_costs
        total_investment = purchase_price + total_purchase_costs

        # 2. Transactional Costs on Resale
        resale_broker_commission = estimated_resale_value * 0.06 # Using 6% as a standard broker fee

        # 3. Opportunity Cost (over 10 months)
        selic_10_months = ((1 + SELIC_RATE_ANNUAL)**(10/12)) - 1
        opportunity_cost = total_investment * selic_10_months

        # 4. Net Profit Calculation
        gross_profit = estimated_resale_value - total_investment
        net_profit = gross_profit - resale_broker_commission - opportunity_cost
            
        adjusted_roi = (net_profit / total_investment) * 100
        
        return adjusted_roi

    except Exception as e:
        logger.error(f"Error in calculate_adjusted_roi: {e}", exc_info=True)
        return 0.0

def setup_logging():
    """Set up logging configuration."""
    logger = logging.getLogger("run_pre_analysis")
    logger.setLevel(logging.INFO)
    
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger

logger = setup_logging()

def impute_leilao_data(df, imputers, logger):
    """Imputes missing values in auction data using pre-trained imputers."""
    df_copy = df.copy()
    logger.info("Imputing missing values in auction data...")

    # Impute n_quartos and n_banheiros using the loaded Linear Regression models
    for col in ['n_quartos', 'n_banheiros']:
        if col not in df_copy.columns:
            logger.warning(f"Column '{col}' not found in auction data. Creating it for imputation.")
            df_copy[col] = np.nan
            
        imputer_model = imputers.get(f'{col}_imputer')
        if imputer_model:
            # Predict only for rows with missing values in the target column but valid 'tamanho'
            missing_mask = df_copy[col].isna() & df_copy['tamanho'].notna()
            if missing_mask.any():
                # The imputer model was trained on 'tamanho'
                features_for_imputation = df_copy.loc[missing_mask, ['tamanho']]
                predicted_values = imputer_model.predict(features_for_imputation)
                df_copy.loc[missing_mask, col] = np.round(predicted_values).clip(1)

    # Impute n_garagem using the saved median value
    if 'n_garagem' not in df_copy.columns:
        df_copy['n_garagem'] = np.nan
    garagem_imputer = imputers.get('n_garagem_imputer')
    if garagem_imputer is not None:
        df_copy['n_garagem'].fillna(garagem_imputer, inplace=True)

    # Fill any remaining NaNs after imputation (e.g., if area_util was NaN) with the median
    for col in ['n_quartos', 'n_banheiros', 'n_garagem']:
        if col in df_copy.columns and df_copy[col].isna().any():
            fallback_median = imputers.get('n_garagem_imputer', 1) # Use garage median or 1 as a fallback
            df_copy[col].fillna(fallback_median, inplace=True)
        # Ensure integer types
        df_copy[col] = df_copy[col].astype(int)
        
    logger.info("Auction data imputation complete.")
    return df_copy

def kb_data_cleaning(df):
    """
    Clean and preprocess the knowledge base dataframe.
    """
    df = df.drop_duplicates(subset=["id"])
    df = df.loc[(df["n_quartos"] > 0) & (df["n_quartos"] < 10)]
    df = df.loc[(df["n_banheiros"] > 0) & (df["n_banheiros"] < 10)]
    df = df.loc[(df["tamanho"] > 10) & (df["tamanho"] < 1000)]
    df = df.loc[(df["preco"] > 10_000) & (df["preco"] < 10_000_000)]
    df = df.loc[df["bairro"].notna()]
    df["bairro"] = df["bairro"].apply(lambda x: unidecode(x.lower().strip()))
    df["preco_m2"] = df["preco"] / df["tamanho"]
    df = df.loc[df["preco_m2"] < 30_000]
    return df

def leilao_data_cleaning(df):
    """
    Clean and preprocess the auction dataframe based on the new schema.
    """
    df_leilao = df.copy()
    df_leilao = df_leilao.applymap(lambda x: None if str(x) == "<NA>" else x)
    df_leilao = df_leilao.drop_duplicates(subset=["id"])

    # Just maintain properties that acept financing (or are not available)
    df_leilao["aceita_financiamento"] = df_leilao["aceita_financiamento"].apply(str).fillna("N/A")
    df_leilao = df_leilao.loc[(df_leilao["aceita_financiamento"] == "True") | (df_leilao["aceita_financiamento"] == "N/A")]

    # Rename columns to match the historical schema used across the script
    df_leilao = df_leilao.rename(columns={
        'preco_primeira_praca': 'preco',
        'area_util': 'tamanho',
        'link_detalhes': 'url'
    })

    # Ensure essential numeric columns are correctly typed, coercing errors to NaN
    for col in ['preco', 'tamanho', 'preco_atual']:
        if col in df_leilao.columns:
            df_leilao[col] = pd.to_numeric(df_leilao[col], errors='coerce')
    
    # Drop rows that are unusable for analysis due to missing core information
    df_leilao.dropna(subset=['preco', 'tamanho', 'bairro', 'preco_atual'], inplace=True)

    # Calculate price per square meter
    df_leilao['preco_m2'] = df_leilao['preco'] / df_leilao['tamanho']

    # Standardize neighborhood names
    df_leilao["bairro"] = df_leilao["bairro"].apply(lambda x: unidecode(str(x).lower().strip()))
    return df_leilao


# Database connection
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

# Select all columns needed for knowledge base
kb_querys = [
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_chaves_na_mao.chaves_na_mao_register",
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_viva_real.viva_real_register",
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_zap_imoveis.zap_imoveis_register"
]

# Fetch knowledge base data
df_kb_list = [con.execute(query).fetch_df() for query in kb_querys]

# Concatenate knowledge base data
df_kb = pd.concat(df_kb_list, ignore_index=True)

# Fetch leilao data
today = datetime.now().strftime("%Y-%m-%d")
df_leilao = con.execute(f"SELECT * FROM kodomiya_leilao_imovel.leilao_imovel_register WHERE data_primeira_praca	> '{today}' OR data_segunda_praca > '{today}'").fetchdf()

logger.info(f"Loaded {len(df_kb)} properties from knowledge base and {len(df_leilao)} open auctions.")


# Data Cleaning
df_kb = kb_data_cleaning(df_kb)
df_leilao = leilao_data_cleaning(df_leilao)

# Impute Lat/Lon for both dataframes using the knowledge base as the source of truth
logger.info("--- Imputing coordinates for Knowledge Base ---")
df_kb = impute_lat_lon(df_kb, df_kb, logger) # Impute KB on itself
logger.info("--- Imputing coordinates for Auction Data ---")
df_leilao = impute_lat_lon(df_leilao, df_kb, logger) # Impute auction data using KB stats


# A. Z-Score Based Analysis (Undervalued Properties)
# ----------------------------------------------------------------
logger.info("Starting Z-Score based analysis...")
df_leilao_zscore = df_leilao.copy()

neighborhood_stats = df_kb.groupby('bairro').agg(
    mean_preco_m2=('preco_m2', 'mean'),
    std_preco_m2=('preco_m2', 'std')
).reset_index()

df_leilao_zscore = df_leilao_zscore.merge(neighborhood_stats, on='bairro', how='left')

# Create a flag for when we fall back to city-wide stats
df_leilao_zscore['z_score_fallback'] = df_leilao_zscore['mean_preco_m2'].isnull()

# Get city-wide stats for fallback
city_mean_preco_m2 = df_kb['preco_m2'].mean()
city_std_preco_m2 = df_kb['preco_m2'].std()

# Log if fallback is happening
if df_leilao_zscore['z_score_fallback'].any():
    num_fallbacks = df_leilao_zscore['z_score_fallback'].sum()
    logger.info(f"{num_fallbacks} properties missing neighborhood stats, using city-wide fallback...")
    logger.info(f"Fallback stats: Mean m² Price=${city_mean_preco_m2:.2f}, Std Dev=${city_std_preco_m2:.2f}")

# Apply fallback for mean and std
df_leilao_zscore['mean_preco_m2'].fillna(city_mean_preco_m2, inplace=True)
df_leilao_zscore['std_preco_m2'].fillna(city_std_preco_m2, inplace=True)
    
# Also, replace std of 0 (which gives infinite z-score) with city-wide std
# This can happen if a neighborhood has only one listing in the KB
df_leilao_zscore.loc[df_leilao_zscore['std_preco_m2'] == 0, 'std_preco_m2'] = city_std_preco_m2

df_leilao_zscore['z_score'] = (
    (df_leilao_zscore['preco_m2'] - df_leilao_zscore['mean_preco_m2']) / 
        df_leilao_zscore['std_preco_m2']
)

undervalued_properties = df_leilao_zscore.copy()
undervalued_properties.loc[:, 'z_score_rank'] = undervalued_properties['z_score'].rank(ascending=True)
undervalued_properties.loc[:, 'estimated_market_value_zscore'] = undervalued_properties['mean_preco_m2'] * undervalued_properties['tamanho']

undervalued_properties['z_score_roi'] = undervalued_properties.apply(
    lambda row: calculate_adjusted_roi(row['preco_atual'], row['estimated_market_value_zscore'], logger),
    axis=1
)

logger.info(f"Found {len(undervalued_properties)} potentially undervalued properties (Z-score).")

all_opportunities = undervalued_properties[['id', 'preco_atual', 'bairro', 'tamanho', 'url', 'rua', 'z_score', 'z_score_rank', 'z_score_roi', 'z_score_fallback']].copy()
    
# Re-populate descriptive data for all opportunities to fix NaNs from outer merges
analysis_cols = ['id', 'z_score', 'z_score_rank', 'z_score_roi', 'z_score_fallback']
detail_cols = ['preco_atual', 'bairro', 'tamanho', 'url', 'rua', 'descricao']

analysis_cols_present = [c for c in analysis_cols if c in all_opportunities.columns]
detail_cols_present = ['id'] + [c for c in detail_cols if c in df_leilao.columns]

all_opportunities = all_opportunities[analysis_cols_present].merge(df_leilao[detail_cols_present], on='id', how='left')
    
all_opportunities['z_score_rank'].fillna(999, inplace=True)
all_opportunities.fillna({'z_score_roi': 0, 'z_score_fallback': False}, inplace=True)

all_opportunities['final_score'] = all_opportunities['z_score_rank'].astype(int)

final_opportunities = all_opportunities[
    (all_opportunities['z_score_rank'] != 999)
].sort_values(by='final_score', ascending=True)

2025-07-04 11:03:54,421 - run_pre_analysis - INFO - Loaded 5168 properties from knowledge base and 89 open auctions.
2025-07-04 11:03:54,443 - run_pre_analysis - INFO - --- Imputing coordinates for Knowledge Base ---
2025-07-04 11:03:54,445 - run_pre_analysis - INFO - Starting lat/lon imputation. Nulls before: Lat=48, Lon=48
2025-07-04 11:03:54,447 - run_pre_analysis - INFO - Using KB for imputation. City-wide mean: Lat=-25.4522, Lon=-49.2720
2025-07-04 11:03:54,453 - run_pre_analysis - INFO - Lat/lon imputation complete. Nulls after: Lat=0, Lon=0
2025-07-04 11:03:54,454 - run_pre_analysis - INFO - --- Imputing coordinates for Auction Data ---
2025-07-04 11:03:54,455 - run_pre_analysis - INFO - Starting lat/lon imputation. Nulls before: Lat=3, Lon=3
2025-07-04 11:03:54,457 - run_pre_analysis - INFO - Using KB for imputation. City-wide mean: Lat=-25.4528, Lon=-49.2722
2025-07-04 11:03:54,464 - run_pre_analysis - INFO - Lat/lon imputation complete. Nulls after: Lat=0, Lon=0
2025-07-04 11

C:\Users\PedroMiyasaki\AppData\Local\Temp\ipykernel_22088\842591247.py:162: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_leilao = df_leilao.applymap(lambda x: None if str(x) == "<NA>" else x)
C:\Users\PedroMiyasaki\AppData\Local\Temp\ipykernel_22088\842591247.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['latitude'].fillna(city_mean_lat, inplace=True)
C:\Users\PedroMiyasaki\AppData\Local\Temp\ipykernel_22088\842591247.py:36: FutureWarning: A value is trying to be set on a cop